# Makers

In this tutorial, you will:

- Learn about `Maker`s.
- Understand how to use a `Maker` to update the parameters of jobs in a flow.


## What is a `Maker`?

A `Maker` is class that makes it convenient to update parameters on-the-fly in a workflow. It is best explained by example.

Let's start by defining a simple `Maker` that either adds or multiplies two numbers together, which we will do twice to make a flow. Note that all classes inheriting from the `Maker` base class must have a `name` variable and a `make` method.


In [21]:
import warnings

warnings.filterwarnings("ignore", "Using `tqdm.autonotebook.tqdm`")

In [22]:
from dataclasses import dataclass
from jobflow import job, Flow, Maker
from jobflow.managers.local import run_locally

@dataclass
class AddMaker(Maker):
    name: str = "Add Maker"
    operation: str = "add"

    @job
    def make(self, a, b):
        if self.operation == "add":
            return a + b
        elif self.operation == "mult":
            return a * b
        else:
            raise ValueError(f"Unknown operation: {self.operation}")


job1 = AddMaker().make(a=2, b=3)
job2 = AddMaker().make(a=job1.output, b=4)

flow = Flow([job1, job2])
responses = run_locally(flow)

2023-06-07 23:15:21,769 INFO Started executing jobs locally
2023-06-07 23:15:21,772 INFO Starting job - Add Maker (0dc27add-a3af-401e-9526-31476d89a175)
2023-06-07 23:15:21,776 INFO Finished job - Add Maker (0dc27add-a3af-401e-9526-31476d89a175)
2023-06-07 23:15:21,777 INFO Starting job - Add Maker (dbf82e6a-7f80-44ce-89a3-1d7ee0834ebd)
2023-06-07 23:15:21,781 INFO Finished job - Add Maker (dbf82e6a-7f80-44ce-89a3-1d7ee0834ebd)
2023-06-07 23:15:21,784 INFO Finished executing jobs locally


In [23]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

0dc27add-a3af-401e-9526-31476d89a175 -> {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
dbf82e6a-7f80-44ce-89a3-1d7ee0834ebd -> {1: Response(output=9, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


Right now, nothing particularly special has happened here. But what if you had a much more complex workflow with many steps and you wanted to change the `AddMaker` keyword arguments, but only for a few individual jobs in the flow? That's where the `Maker` comes in handy. Let's see how it works.


In [24]:
from dataclasses import dataclass
from jobflow import job, Flow, Maker
from jobflow.managers.local import run_locally

@dataclass
class AddMaker(Maker):
    name: str = "Add Maker"
    operation: str = "add"

    @job
    def make(self, a, b):
        if self.operation == "add":
            return a + b
        elif self.operation == "mult":
            return a * b
        else:
            raise ValueError(f"Unknown operation: {self.operation}")

@dataclass
class SubtractMaker(Maker):
    name: str = "Subtract Maker"

    @job
    def make(self, a, b):
        return b - a

job1 = AddMaker().make(a=2, b=3)
job2 = SubtractMaker().make(a=job1.output, b=4)
flow = Flow([job1, job2])

In [25]:
flow.update_maker_kwargs({"operation": "mult"}, name_filter="Add Maker")
responses = run_locally(flow)

2023-06-07 23:15:22,081 INFO Started executing jobs locally
2023-06-07 23:15:22,083 INFO Starting job - Add Maker (3a00f782-d462-4ece-afc1-878c393410a5)
2023-06-07 23:15:22,084 INFO Finished job - Add Maker (3a00f782-d462-4ece-afc1-878c393410a5)
2023-06-07 23:15:22,087 INFO Starting job - Subtract Maker (9bfbd1cb-a62f-48d6-ae86-ebf0e320922c)
2023-06-07 23:15:22,089 INFO Finished job - Subtract Maker (9bfbd1cb-a62f-48d6-ae86-ebf0e320922c)
2023-06-07 23:15:22,091 INFO Finished executing jobs locally


In [26]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

3a00f782-d462-4ece-afc1-878c393410a5 -> {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
9bfbd1cb-a62f-48d6-ae86-ebf0e320922c -> {1: Response(output=-2, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


In this example, we have updated the keyword arguments ("kwargs") of jobs in a flow using a `name_filter` and the `update_maker_kwargs` function, which functions because the classes in the flow are themselves `Maker` objects.

Of course, we could have simply done `job1 = AddMaker(operation="mult").make(a=2, b=3)` to begin with, but in practice if you were to have instead impotred this flow from some external Python package, you might not be able to modify the `AddMaker` class directly. In this case, the `Maker` class provides a convenient way to update the parameters of jobs in a flow without having to redefine the flow itself.
